In [1]:
# https://docs.fast.ai/text.html

## Quick Start: Training an IMDb sentiment model with ULMFiT
Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps:

1. Reading and viewing the IMDb data
2. Getting your data ready for modeling
3. Fine-tuning a language model
4. Building a classifier
### Reading and viewing the IMDb data
First let's import everything we need for text.

In [1]:
from fastai.text import * 

Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do is to preprocess our data so that we change the raw texts to lists of words, or tokens (a step that is called tokenization) then transform these tokens into numbers (a step that is called numericalization). These numbers are then passed to embedding layers that will convert them in arrays of floats before passing them through a model.

You can find on the web plenty of __Word Embeddings__ to directly convert your tokens into floats. Those word embeddings have generally be trained on a large corpus such as wikipedia. Following the work of __ULMFiT__, the fastai library is more focused on using pre-trained Language Models and fine-tuning them. Word embeddings are just vectors of 300 or 400 floats that represent different words, but a pretrained language model not only has those, but has also been trained to get a representation of full sentences and documents.

That's why the library is structured around three steps:

1. Get your data preprocessed and ready to use in a minimum amount of code,
2. Create a language model with pretrained weights that you can fine-tune to your dataset,
3. Create other models such as classifiers on top of the encoder of the language model.
To show examples, we have provided a small sample of the <a href="https://www.imdb.com/interfaces/">IMDB dataset</a> which contains 1,000 reviews of movies with labels (positive or negative).

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/rossleecooloh/.fastai/data/imdb_sample')

Creating a dataset from your raw texts is very simple if you have it in one of those ways

- organized it in folders in an ImageNet style
- organized in a csv file with labels columns and a text columns

Here, the sample from imdb is in a texts csv files that looks like this:

In [3]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


### Getting your data ready for modeling
To get a `DataBunch` quickly, there are also several factory methods depending on how our data is structured. They are all detailed in `text.data`, here we'll use the method `from_csv` of the `TextLMDataBunch` (to get the data ready for a language model) and `TextClasDataBunch` (to get the data ready for a text classifier) classes.

In [6]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

This does all the necessary preprocessing behind the scene. For the classifier, we also pass the vocabulary (mapping from ids to words) that we want to use: this is to ensure that `data_clas` will use the same dictionary as `data_lm`.

Since this step can be a bit time-consuming, it's best to save the result with:

In [5]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

NameError: name 'data_lm' is not defined

In [7]:
path

PosixPath('/home/rossleecooloh/.fastai/data/imdb_sample')

In [8]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

Note that you can load the data with different `DataBunch` parameters (batch size, `bptt`,...)

### Fine-tuning a language model
We can use the `data_lm` object we created earlier to fine-tune a pretrained language model. fast.ai has an English model with an AWD-LSTM architecture available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.

In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
# learn.freeze()
learn.fit_one_cycle(1, 1e-2)  # not unfreeze yet

"""
default = freeze

Freeze up to last layer group.

Sets every layer group except the last to untrainable 
(i.e. requires_grad=False).
"""

epoch,train_loss,valid_loss,accuracy,time
0,4.303236,3.905817,0.279673,07:40


'\ndefault = freeze\n\nFreeze up to last layer group.\n\nSets every layer group except the last to untrainable \n(i.e. requires_grad=False).\n'

In [13]:
learn.summary()  # unfreeze前，只有一層有被train

SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 400]            0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
Linear               [70, 8872]           3,557,672  True      
______________________________________________________________________
RNNDropout           [70, 400]            0          False     
______________________________________________________________________

Total params: 3,557,672
Total trainable params: 3,557,672
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss func

In [18]:
learn.freeze()
learn.summary()

SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 400]            0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
Linear               [70, 8872]           3,557,672  True      
______________________________________________________________________
RNNDropout           [70, 400]            0          False     
______________________________________________________________________

Total params: 3,557,672
Total trainable params: 3,557,672
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss func

Like a computer vision model, we can then unfreeze the model and fine-tune it.

In [8]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)
learn.summary()

epoch,train_loss,valid_loss,accuracy,time
0,3.931772,3.843006,0.290699,07:11


SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 400]            0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
Linear               [70, 8872]           3,557,672  True      
______________________________________________________________________
RNNDropout           [70, 400]            0          False     
______________________________________________________________________

Total params: 3,557,672
Total trainable params: 3,557,672
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss func

To evaluate your language model, you can run the `Learner.predict` method and specify the number of words you want it to guess.

In [9]:
# language model prediction
learn.predict("This is a review about", n_words=20)

'This is a review about lame harvey cars such as 3 ed mckenna & Diet Selection and the DREAMSCAPES Circle ,'

It doesn't make much sense (we have a tiny vocabulary here and didn't train much on it) but note that it respects basic grammar (which comes from the pretrained model).

Finally we save the encoder to be able to use it for classification in the next section.

In [10]:
# the encoder of language model
learn.save_encoder('fine_tuned_encoder')

### Building a classifier
We now use the `data_clas` object we created earlier to build a classifier with our fine-tuned encoder. The learner object can be done in a single line.

In [27]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_encoder')

In [28]:
data_clas.show_batch()

text,target
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk xxunk experience in general . xxmaj with graphics",positive
"xxbos xxmaj how viewers react to this new "" adaption "" of xxmaj shirley xxmaj jackson 's book , which was promoted as xxup not being a remake of the original 1963 movie ( true enough ) , will be based , i suspect , on the following : those who were big fans of either the book or original movie are not going to think much of this one",negative
"xxbos xxmaj the trouble with the book , "" xxmaj memoirs of a xxmaj geisha "" is that it had xxmaj japanese surfaces but underneath the surfaces it was all an xxmaj american man 's way of thinking . xxmaj reading the book is like watching a magnificent ballet with great music , sets , and costumes yet performed by xxunk animals dressed in those xxunk far from xxmaj japanese",negative


In [30]:
learn.freeze()
learn.summary()

SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [33, 400]            0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
BatchNorm1d          [1200]               2,400      True      
______________________________________________________________________
Dropout              [1200]               0          False     
______________________________________________________________________
Linear               [50]                 60,050     True      
______________________________________________________________________
ReLU                 [50]                 0          False     
________________________________________________

In [31]:
# 不太懂這邊是怎麼訓練的? 在參數都固定的情況等於直接model predict?
# 還是多加了task-specific層上去微調?
learn.fit_one_cycle(cyc_len=1, max_lr=1e-2)
learn.summary()

epoch,train_loss,valid_loss,accuracy,time
0,0.576525,0.522008,0.820895,30:09


SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [33, 400]            0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
BatchNorm1d          [1200]               2,400      True      
______________________________________________________________________
Dropout              [1200]               0          False     
______________________________________________________________________
Linear               [50]                 60,050     True      
______________________________________________________________________
ReLU                 [50]                 0          False     
________________________________________________

Again, we can unfreeze the model and fine-tune it.

In [17]:
learn.freeze_to(-2)  ## Unfreeze最後一個layer + 分類layer
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))
learn.summary()

epoch,train_loss,valid_loss,accuracy,time
0,0.522795,0.420133,0.820895,04:53


SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [33, 400]            0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
BatchNorm1d          [1200]               2,400      True      
______________________________________________________________________
Dropout              [1200]               0          False     
______________________________________________________________________
Linear               [50]                 60,050     True      
______________________________________________________________________
ReLU                 [50]                 0          False     
________________________________________________

In [18]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))
learn.summary()

epoch,train_loss,valid_loss,accuracy,time
0,0.400513,0.357523,0.835821,15:27


SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [33, 400]            0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
RNNDropout           [33, 1152]           0          False     
______________________________________________________________________
BatchNorm1d          [1200]               2,400      True      
______________________________________________________________________
Dropout              [1200]               0          False     
______________________________________________________________________
Linear               [50]                 60,050     True      
______________________________________________________________________
ReLU                 [50]                 0          False     
________________________________________________

Again, we can predict on a raw text by using the __Learner.predict__ method.

In [19]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.0062, 0.9938]))